In [11]:
import requests
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, Table, MetaData
from sqlalchemy.orm import scoped_session, sessionmaker

# E

In [2]:
df_vagas = pd.read_csv("../stone2/data/jobs.csv", sep=";") 
df_inscritos = pd.read_csv("../stone2/data/applications.csv", sep=";")

# T

## Functions

In [7]:
## functions.py
def cleanJob(col):
    reps = {
    'valores_validos': ['Consultor Comercial Externo',
                        'Consultor Comercial',
                        'Agente Stone'],
    'valor_reposicao': np.nan
    }
    col = col.replace('Consultor Comercial  Externo', 'Consultor Comercial Externo')
    col = col.where(col.isin(reps['valores_validos']), reps['valor_reposicao'])
    return col

getCity = lambda lista: lista[-1].strip()
getJob = lambda lista: lista[-2].strip()
    

## Enriquece cidade

In [8]:
splitted_job = df_vagas.job_name.str.split("-")
df_vagas['cidade'] = [getCity(row) for row in splitted_job]
df_vagas['cargo'] = [getJob(row) if len(row) > 1 else None for row in splitted_job]

## Limpa job

In [9]:
df_vagas['cargo'] = cleanJob(df_vagas['cargo'])

## Merge enriquecimento

In [10]:
df_inscritosVagas = df_inscritos.merge(df_vagas, left_on='job_id', right_on='job_id').drop(columns=['job_name'])

# L

## Configura conexão

In [14]:
db = 'postgresql+psycopg2'
user = 'airflow'
pwd = 'airflow'
host = '0.0.0.0'
port = '5432'
db_name = 'airflow'

db_engine = create_engine(f"{db}://{user}:{pwd}@{host}:{port}/{db_name}")
db_engine.connect()

## Insere no PstgreSQL

In [15]:
df_inscritosVagas.to_sql("stone", db_engine, if_exists='replace')#if_exists='append' em caso de rotina 